In [1]:
from typing import List
from itertools import accumulate
from time import perf_counter_ns
import numpy as np
from scipy.sparse import csc_matrix, kron, identity
import cirq
import openfermion as of
from openfermionpyscf import run_pyscf
import quimb.tensor as qtn
from quimb.tensor.tensor_1d import MatrixProductState, MatrixProductOperator
from convert import to_groups_of
from error_pert import get_v2_sarray
from qpe_trotter import v2_pauli_sum, v2_qubop
from kcommute import get_si_sets
from tensor_network_common import pauli_sum_to_mpo, mps_to_vector

In [2]:
molec = "LiH"
basis = "sto-3g"
n_elec = 4
geometry = of.chem.geometry_from_pubchem(molec)
multiplicity = 1
molecule = of.chem.MolecularData(
    geometry, basis, multiplicity
)
molecule = run_pyscf(molecule, run_scf=1, run_fci=1)
print(f"HF energy:", molecule.hf_energy)
print(f"FCI energy:", molecule.fci_energy)
hamiltonian = molecule.get_molecular_hamiltonian()
hamiltonian_qubop = of.transforms.jordan_wigner(hamiltonian)
hamiltonian_psum = of.transforms.qubit_operator_to_pauli_sum(hamiltonian_qubop)

nq = of.utils.count_qubits(hamiltonian_qubop)
nterms = len(hamiltonian_qubop.terms)
print(f"Hamiltonian has {nq} qubits and {nterms} terms.")

HF energy: -7.767362135748557
FCI energy: -7.784460280031223
Hamiltonian has 12 qubits and 631 terms.


In [3]:
qs = cirq.LineQubit.range(nq)
hamiltonian_mpo = pauli_sum_to_mpo(hamiltonian_psum, qs, 100)
dmrg = qtn.DMRG(hamiltonian_mpo, bond_dims=15)
converged = dmrg.solve()
if not converged:
    print("DMRG did not converge.")
ground_state = dmrg.state
ground_state_vec = mps_to_vector(ground_state)

sorted_inds = ['k0', 'k1', 'k2', 'k3', 'k4', 'k5', 'k6', 'k7', 'k8', 'k9', 'k10', 'k11']


/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/cotengra/hyperoptimizers/hyper.py:55: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization. It is recommended to install one of these libraries for higher quality contraction paths.
  warnings.warn(
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: divide by zero encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: overflow encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: invalid value encountered in matmul
  return func(*args, **kwargs)
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/autoray/autoray.py:81: RuntimeWarning: divide by zero encountered in 

In [4]:
groups = get_si_sets(hamiltonian_psum, nq)
print(f"There are {len(groups)} groups.")

There are 42 groups.


## Sparse matrices

In [5]:
# Use the code from the paper.
start_time = perf_counter_ns()
group_qubops = to_groups_of(groups)
sparse_frag_ops = []
# Convert group operators to sparse matrices.
# We must make sure the matrices have all the same size.
# If a matrix is not big enough, tensor it with I on the right.
for op in group_qubops:
    nq_op = of.utils.count_qubits(op)
    op_sparse = of.linalg.get_sparse_operator(op)
    if nq_op != nq:
        eye_diff = identity(2 ** (nq - nq_op), dtype="complex", format='csc')
        new_op = kron(op_sparse, eye_diff, format="csc")
        sparse_frag_ops.append(new_op)
    else:
        sparse_frag_ops.append(op_sparse)
v2_sparse = get_v2_sarray(sparse_frag_ops)
eps2 = np.vdot(ground_state_vec, v2_sparse @ ground_state_vec).real
end_time = perf_counter_ns()
elapsed_time = end_time - start_time
print(f"Got eps2={eps2} in {elapsed_time:4.5e} ns")

Got eps2=0.009771387704065805 in 1.34142e+09 ns


## `cirq.PauliSum`

In [ ]:
start_time = perf_counter_ns()
group_psums = [sum(group) for group in groups]
v2_psum = v2_pauli_sum(group_psums)
qubit_map = {q: i for i, q in enumerate(qs)}
eps2_psum = v2_psum.expectation_from_state_vector(ground_state_vec, qubit_map)
end_time = perf_counter_ns()
elapsed_time = end_time - start_time
print(f"Got eps2={eps2_psum} in {elapsed_time:4.5e} ns")

Got eps2=(0.009773324395417103-3.445143113209419e-20j) in 4.81487e+11 ns


## `of.QubitOperator`

In [ ]:
start_time = perf_counter_ns()
group_qubops = to_groups_of(groups)
v2_op = v2_qubop(group_psums)
qubit_map = {q: i for i, q in enumerate(qs)}
eps2_qubop = v2_psum.expectation_from_state_vector(ground_state_vec, qubit_map)
end_time = perf_counter_ns()
elapsed_time = end_time - start_time
print(f"Got eps2={eps2_qubop} in {elapsed_time:4.5e} ns")

Got eps2=(0.009773324395417103-3.445143113209419e-20j) in 5.01746e+11 ns


## MPO

In [6]:
def to_groups_mpo(groups: List[List[cirq.PauliString]], qs: List[cirq.Qid], max_bond: int) -> List[MatrixProductOperator]:
    """Convert groups from SI into a list of MatrixProductOperators."""

    mpos: List[MatrixProductOperator] = []
    for group in groups:
        group_psum = sum(group)
        group_mpo = pauli_sum_to_mpo(group_psum, qs, max_bond=max_bond)
        mpos.append(group_mpo.copy())
    return mpos

def mpo_add_compress(
    mpo_a: MatrixProductOperator, mpo_b: MatrixProductOperator,
    compress: bool = False, max_bond: int = 100
) -> MatrixProductOperator:
    """Add two MPOs, optionally compressing them."""

    mpo_sum = mpo_a + mpo_b
    if compress:
        mpo_sum.compress(max_bond=max_bond)
    return mpo_sum


def mpo_commutator(mpo_a: MatrixProductOperator, mpo_b: MatrixProductOperator) -> MatrixProductOperator:
    """Take the commutator of two MPOs."""

    return mpo_a.apply(mpo_b) - mpo_b.apply(mpo_a)


def zeros_mpo(nsites: int, phys_dim: int=2) -> MatrixProductOperator:
    """Returns an MPO corresponding to a matirx of all zeros."""

    def fill_fun(shape):
        return np.zeros(shape, dtype=complex)
    
    return MatrixProductOperator.from_fill_fn(fill_fun, L=nsites, bond_dim=1, phys_dim=phys_dim)


def get_v2_contrib_mpo(fragments_list: List[MatrixProductOperator], psi: MatrixProductState, max_bond: int) -> float:
    """Calculate eps2 when the fragments are MPOs."""

    max_mpo_sites = max([len(mpo.tensor_map) for mpo in fragments_list])

    frags_len = len(fragments_list)
    frag_sums_l2r = list(accumulate(fragments_list, lambda a, b: mpo_add_compress(a, b, True, max_bond)))
    temp = reversed(fragments_list)
    frag_sums_r2l = list(accumulate(temp, lambda a, b: mpo_add_compress(a, b, True, max_bond)))
    frag_sums_r2l = list(reversed(frag_sums_r2l))
    frag_sums_r2l.append(zeros_mpo(max_mpo_sites))
    frag_combs_V1_v2 = [(frag_sums_l2r[i-1], fragments_list[i], frag_sums_r2l[i+1]) for i in range (1, frags_len)]
    eps2 = 0
    for i,j,k in frag_combs_V1_v2:
        V1_term = mpo_commutator(i, j)
        term1 = psi.H @ psi.gate_with_mpo(mpo_commutator(V1_term, k))
        term2 = psi.H @ psi.gate_with_mpo(mpo_commutator(V1_term, j))
        eps2 += - term1 *1/3 - term2 *1/6
    return eps2.real

In [13]:
max_mpo_bond = 5
start_time = perf_counter_ns()
mpo_fragments = to_groups_mpo(groups, qs, max_mpo_bond)
end_time = perf_counter_ns()
elapsed_time_convert = end_time - start_time

In [14]:
start_time = perf_counter_ns()
eps2_mpo = get_v2_contrib_mpo(mpo_fragments, ground_state, max_mpo_bond)
end_time = perf_counter_ns()
elapsed_time_calculate = end_time - start_time

print(f"Got eps2={eps2_mpo}.\nConversion time: {elapsed_time_convert}, calculation time: {elapsed_time_calculate}.")

Got eps2=0.009230812660028556.
Conversion time: 1230262625, calculation time: 82577310625.
